In [2]:
from datetime import datetime
def ip_to_int(ip_address):
    octets = ip_address.split('.')
    int_ip = (int(octets[0]) << 24) + (int(octets[1]) << 16) + (int(octets[2]) << 8) + int(octets[3])
    return int_ip



def datetime_to_timestamp(date_str):
    date_format = "%d-%m-%y %H:%M"
    dt = datetime.strptime(date_str, date_format)
    timestamp = int(dt.timestamp())
    return timestamp

In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [4]:
file=r"C:\Users\chira\Desktop\IDS\SensorNetGuard A Dataset for Identifying Malicious Sensor Nodes.csv"
df= pd.read_csv(file)

df.head(5)

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,1,01-02-23 0:00,192.168.119.138,52.018229,2.727317,0.955687,130.967180,-52.059232,23.974265,82.982598,...,3,11.323044,11.458199,20.670539,14.385439,0.109146,26.376693,29.186341,76.811986,0
1,2,01-02-23 0:01,192.168.225.56,59.504648,1.435058,1.228460,82.634358,-57.754723,22.572052,66.734853,...,6,8.277362,8.832980,15.555122,14.316681,0.077136,34.963429,34.203177,112.495912,0
2,3,01-02-23 0:02,192.168.187.210,34.118204,1.320905,1.015245,61.649928,-52.399037,15.667076,74.593605,...,5,8.196374,8.330227,19.470748,11.792084,0.052993,48.934585,31.682662,117.526109,0
3,4,01-02-23 0:03,192.168.78.245,39.643055,1.988791,0.784562,69.704181,-53.727443,20.286962,76.710564,...,6,10.683766,6.525032,16.228955,8.895409,0.087810,47.045600,37.470027,123.058913,0
4,5,01-02-23 0:04,192.168.233.229,40.212679,1.005121,1.412193,123.445612,-49.696620,24.318538,84.477890,...,7,11.176823,10.617432,12.257423,10.168739,0.068823,33.394354,26.582970,71.696911,0


In [5]:
df.describe()

,Node_ID,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,Energy_Consumption_Rate,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,51.178372,2.085671,1.000137,97.413849,-49.465104,19.488316,78.885456,5.251356,4.477100,10.192459,10.197013,15.418190,15.371266,0.105077,40.779507,30.385238,99.578820,0.048700
std,2886.89568,11.611810,0.674931,0.199463,22.437926,5.441244,5.422291,10.879901,1.522230,2.002068,2.328291,2.371997,3.443820,3.479567,0.030656,10.875090,5.447610,20.081485,0.215251
min,1.00000,15.615783,-0.273674,0.212105,11.519978,-69.574136,-3.666482,31.440133,1.483981,-2.000000,2.276865,2.910179,3.661760,3.372971,0.026954,-0.890490,10.265487,24.044248,0.000000
25%,2500.75000,43.626968,1.673938,0.864360,83.309144,-53.111602,16.053767,71.878990,4.372082,3.000000,8.675583,8.663843,13.116899,13.040494,0.087307,33.321420,26.748664,86.044026,0.000000
50%,5000.50000,50.461842,2.021277,0.998534,98.771228,-49.702346,19.663671,79.141462,5.074718,4.000000,10.057934,10.046503,15.241487,15.144926,0.101497,40.460401,30.174862,99.711069,0.000000
75%,7500.25000,57.693898,2.391480,1.132396,112.534272,-45.983668,23.120609,86.252124,5.821342,6.000000,11.507392,11.539378,17.329160,17.310426,0.116259,47.607252,33.777577,113.350059,0.000000
max,10000.00000,113.825524,6.839591,1.750646,175.846021,-24.428489,38.561371,119.009948,15.309509,12.000000,22.926287,26.380494,35.247997,33.894688,0.345463,91.038421,57.655692,169.797139,1.000000


In [6]:
df.Is_Malicious.value_counts()
df['Is_Malicious'] = df['Is_Malicious'].apply(lambda x: 'No' if x == 0 else 'Yes')
df.head(10)

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,1,01-02-23 0:00,192.168.119.138,52.018229,2.727317,0.955687,130.967180,-52.059232,23.974265,82.982598,...,3,11.323044,11.458199,20.670539,14.385439,0.109146,26.376693,29.186341,76.811986,No
1,2,01-02-23 0:01,192.168.225.56,59.504648,1.435058,1.228460,82.634358,-57.754723,22.572052,66.734853,...,6,8.277362,8.832980,15.555122,14.316681,0.077136,34.963429,34.203177,112.495912,No
2,3,01-02-23 0:02,192.168.187.210,34.118204,1.320905,1.015245,61.649928,-52.399037,15.667076,74.593605,...,5,8.196374,8.330227,19.470748,11.792084,0.052993,48.934585,31.682662,117.526109,No
3,4,01-02-23 0:03,192.168.78.245,39.643055,1.988791,0.784562,69.704181,-53.727443,20.286962,76.710564,...,6,10.683766,6.525032,16.228955,8.895409,0.087810,47.045600,37.470027,123.058913,No
4,5,01-02-23 0:04,192.168.233.229,40.212679,1.005121,1.412193,123.445612,-49.696620,24.318538,84.477890,...,7,11.176823,10.617432,12.257423,10.168739,0.068823,33.394354,26.582970,71.696911,No
5,6,01-02-23 0:05,192.168.230.20,42.539308,1.777329,1.226797,140.306629,-48.164891,22.562783,80.792853,...,3,9.100424,10.912167,17.039896,17.098904,0.095630,41.592922,37.011924,135.173141,No
6,7,01-02-23 0:06,192.168.201.178,34.371305,2.122728,1.238309,82.759149,-46.749541,22.199859,83.896476,...,5,5.514884,9.764031,14.729525,14.537270,0.112991,51.911647,33.413317,113.722314,No
7,8,01-02-23 0:07,192.168.0.73,39.421713,1.632756,1.070672,128.744279,-47.965449,22.017101,73.018853,...,3,8.785059,7.424502,14.028956,16.584326,0.150733,43.373535,32.514071,101.301618,No
8,9,01-02-23 0:08,192.168.35.14,64.392638,1.771348,0.614431,105.554913,-45.030005,19.406187,100.505321,...,4,9.652375,9.578266,20.649000,13.220874,0.133769,41.662683,36.024818,130.214575,No
9,10,01-02-23 0:09,192.168.94.186,71.586404,2.874885,1.175556,109.901877,-43.949902,27.354364,76.642262,...,3,8.413089,14.570255,15.308871,14.962364,0.088105,39.467362,31.086755,130.360976,No


In [7]:
data=df.copy()


In [8]:
data["IP_Address"] = data["IP_Address"].apply(
    lambda x: ip_to_int(x))

data["Timestamp"] = data["Timestamp"].apply(
    lambda x: datetime_to_timestamp(x))

In [9]:
numeric_features = data.dtypes[data.dtypes != 'object'].index
scaler = QuantileTransformer() 
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [10]:
data.head()

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,0.0000,0.0000,0.474839,0.557149,0.889185,0.414509,0.941004,0.318241,0.799840,0.642791,...,0.237738,0.723723,0.737099,0.941507,0.398746,0.637313,0.084958,0.424844,0.131058,No
1,0.0001,0.0001,0.882885,0.797457,0.127321,0.873882,0.239220,0.056884,0.714148,0.128127,...,0.772773,0.192712,0.277150,0.540533,0.390394,0.123780,0.300326,0.770404,0.735583,No
2,0.0002,0.0002,0.734026,0.052800,0.085223,0.534389,0.069008,0.291497,0.228055,0.333746,...,0.600601,0.181943,0.200853,0.896742,0.133092,0.009316,0.787897,0.614411,0.817809,No
3,0.0003,0.0003,0.309344,0.146878,0.473462,0.140407,0.112481,0.215304,0.546011,0.407981,...,0.772773,0.616361,0.046128,0.626748,0.017899,0.257606,0.733748,0.902932,0.880012,No
4,0.0004,0.0004,0.918029,0.160557,0.022267,0.980939,0.887481,0.500535,0.816879,0.694722,...,0.891391,0.700736,0.603218,0.163758,0.049857,0.057329,0.251544,0.238196,0.082610,No


In [11]:
data[numeric_features] = data[numeric_features].apply(
    lambda x: (x*255))

data.head()

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,0.000000,0.000000,121.083895,142.072959,226.742072,105.699847,239.956133,81.151438,203.959094,163.911612,...,60.623123,184.549453,187.960149,240.084205,101.680240,162.514775,21.664272,108.335245,33.419896,No
1,0.025503,0.025503,225.135651,203.351592,32.466838,222.839815,61.001162,14.505520,182.107772,32.672382,...,197.057057,49.141638,70.673169,137.835934,99.550370,31.563930,76.583234,196.453006,187.573776,No
2,0.051005,0.051005,187.176520,13.463902,21.731946,136.269308,17.596966,74.331793,58.154097,85.105315,...,153.153153,46.395358,51.217478,228.669333,33.938384,2.375649,200.913646,156.674873,208.541312,No
3,0.076508,0.076508,78.882768,37.453888,120.732848,35.803749,28.682707,54.902602,139.232799,104.035240,...,197.057057,157.172011,11.762590,159.820777,4.564352,65.689419,187.105682,230.247645,224.403065,No
4,0.102010,0.102010,234.097299,40.942101,5.678140,250.139447,226.307611,127.636374,208.304062,177.154120,...,227.304805,178.687785,153.820673,41.758193,12.713602,14.618841,64.143701,60.740107,21.065528,No


In [12]:
df0=data[data['Is_Malicious']=='No'].drop(['Is_Malicious'],axis=1)
df1=data[data['Is_Malicious']=='Yes'].drop(['Is_Malicious'],axis=1)


In [13]:
df0.head(10)

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,Energy_Consumption_Rate,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth
0,0.000000,0.000000,121.083895,142.072959,226.742072,105.699847,239.956133,81.151438,203.959094,163.911612,65.568566,60.623123,184.549453,187.960149,240.084205,101.680240,162.514775,21.664272,108.335245,33.419896
1,0.025503,0.025503,225.135651,203.351592,32.466838,222.839815,61.001162,14.505520,182.107772,32.672382,101.983856,197.057057,49.141638,70.673169,137.835934,99.550370,31.563930,76.583234,196.453006,187.573776
2,0.051005,0.051005,187.176520,13.463902,21.731946,136.269308,17.596966,74.331793,58.154097,85.105315,110.348090,153.153153,46.395358,51.217478,228.669333,33.938384,2.375649,200.913646,156.674873,208.541312
3,0.076508,0.076508,78.882768,37.453888,120.732848,35.803749,28.682707,54.902602,139.232799,104.035240,23.774116,197.057057,157.172011,11.762590,159.820777,4.564352,65.689419,187.105682,230.247645,224.403065
4,0.102010,0.102010,234.097299,40.942101,5.678140,250.139447,226.307611,127.636374,208.304062,177.154120,166.162357,227.304805,178.687785,153.820673,41.758193,12.713602,14.618841,64.143701,60.740107,21.065528
5,0.127513,0.127513,230.104492,55.628220,81.187760,222.188020,249.713238,155.459160,181.949321,143.089993,9.394915,60.623123,81.427556,166.464968,183.575678,186.687764,99.315893,138.288980,226.721698,245.663993
6,0.153015,0.153015,201.034773,14.164554,147.225758,225.311699,61.435567,180.425741,175.439109,172.040914,239.860954,153.153153,3.303646,113.764885,111.727392,106.318343,178.693196,218.010659,185.826703,192.726429
7,0.178518,0.178518,0.237921,35.880296,57.197906,162.915019,236.807744,159.070005,172.041261,72.416588,146.483636,60.623123,67.740107,27.084409,90.036778,172.894445,242.355567,155.331053,171.616702,135.140359
8,0.204020,0.204020,35.081496,227.521215,80.312323,6.902967,160.846275,203.445786,122.086604,250.167079,161.580609,104.399399,107.144813,105.390909,239.880129,68.226867,232.036478,139.144405,218.013827,238.914454
9,0.229523,0.229523,95.628990,243.839532,234.997616,206.821731,180.837956,216.243587,238.053798,103.461196,149.231708,60.623123,54.035188,245.718331,129.751908,120.861943,67.054685,118.166841,145.284216,239.134383


In [21]:
image_path = "train/0/"
os.makedirs(image_path)

count = 0
ims = []

for i in range(len(df0)):  
    im = df0.iloc[i].values
    ims = np.append(ims, im)
    count += len(im)

    if count >= 9 * 9 * 3:  # For an RGB image of size 9x9, we need 9*9*3 elements
        ims = ims[:9 * 9 * 3]  # Trim the ims to exactly 9*9*3 if it exceeds
        ims = np.array(ims).reshape(9, 9, 3)  # Reshape to 9x9x3
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array, mode='RGB')  # 'RGB' mode for color image
        new_image.save(os.path.join(image_path, f"{i}.png"))
        count = 0
        ims = []

# If there are remaining ims, process them as well
if count > 0:
    ims = np.pad(ims, (0, 9 * 9 * 3 - len(ims)), 'constant', constant_values=0)  # Padding to make a complete 9x9x3 array
    ims = np.array(ims).reshape(9, 9, 3)
    array = np.array(ims, dtype=np.uint8)
    new_image = Image.fromarray(array, mode='RGB')
    new_image.save(os.path.join(image_path, f"{len(df0)}.png"))

In [22]:
image_path = "train/1/"
os.makedirs(image_path)

count = 0
ims = []

for i in range(len(df1)):  
    im = df1.iloc[i].values
    ims = np.append(ims, im)
    count += len(im)

    if count >= 9 * 9 * 3:  # For an RGB image of size 9x9, we need 9*9*3 elements
        ims = ims[:9 * 9 * 3]  # Trim the ims to exactly 9*9*3 if it exceeds
        ims = np.array(ims).reshape(9, 9, 3)  # Reshape to 9x9x3
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array, mode='RGB')  # 'RGB' mode for color image
        new_image.save(os.path.join(image_path, f"{i}.png"))
        count = 0
        ims = []

# If there are remaining ims, process them as well
if count > 0:
    ims = np.pad(ims, (0, 9 * 9 * 3 - len(ims)), 'constant', constant_values=0)  # Padding to make a complete 9x9x3 array
    ims = np.array(ims).reshape(9, 9, 3)
    array = np.array(ims, dtype=np.uint8)
    new_image = Image.fromarray(array, mode='RGB')
    new_image.save(os.path.join(image_path, f"{len(df0)}.png"))

In [23]:
Train_Dir='./train/'
Val_Dir='./test/'
allimgs=[]
for subdir in os.listdir(Train_Dir):
    for filename in os.listdir(os.path.join(Train_Dir,subdir)):
        filepath=os.path.join(Train_Dir,subdir,filename)
        allimgs.append(filepath)
print(len(allimgs)) # Print the total number of images

770


In [26]:
Numbers=len(allimgs)//2	#size of test set (20%)

def mymovefile(srcfile,dstfile):
    if not os.path.isfile(srcfile):
        print ("%s not exist!"%(srcfile))
    else:
        fpath,fname=os.path.split(dstfile)    
        if not os.path.exists(fpath):
            os.makedirs(fpath)               
        shutil.move(srcfile,dstfile)          
        #print ("move %s -> %s"%(srcfile,dstfile))

Numbers


385

In [27]:
val_imgs=random.sample(allimgs,Numbers)
for img in val_imgs:
    dest_path=img.replace(Train_Dir,Val_Dir)
    mymovefile(img,dest_path)
print('Finish creating test set')

Finish creating test set


In [28]:
def get_224(folder,dstdir):
    imgfilepaths=[]
    for root,dirs,imgs in os.walk(folder):
        for thisimg in imgs:
            thisimg_path=os.path.join(root,thisimg)
            imgfilepaths.append(thisimg_path)
    for thisimg_path in imgfilepaths:
        dir_name,filename=os.path.split(thisimg_path)
        dir_name=dir_name.replace(folder,dstdir)
        new_file_path=os.path.join(dir_name,filename)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        img=cv2.imread(thisimg_path)
        img=cv2.resize(img,(224,224))
        cv2.imwrite(new_file_path,img)
    print('Finish resizing'.format(folder=folder))

In [29]:
DATA_DIR2_224='./test_224/'
get_224(folder='./test/',dstdir=DATA_DIR2_224)

Finish resizing


In [30]:
DATA_DIR_224='./train_224/'
get_224(folder='./train/',dstdir=DATA_DIR_224)

Finish resizing
